In [ ]:
!pip install tsplib95

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.5 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0:
      Successfully uninstalled tabulate-0.9.0
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.21.0 requires tabulate>=0.9, but you have tabulate 0.8.10 which is incompatible.


In [ ]:
# Run 1: population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=2, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3194.8952732592616
Generation 2: Best Cost = 3194.8952732592616
Generation 3: Best Cost = 3194.8952732592616
Generation 4: Best Cost = 3194.8952732592616
Generation 5: Best Cost = 3194.8952732592616
Generation 6: Best Cost = 3194.8952732592616
Best solution found: [ 9 35 53 38 62 49 52 22 26  5 25 67 61 15 24  6 29  3 19  8 60 27 16 10
 20 66  2 23 13 31 12 58 68 32 40  1 11 48 54 51 55  4 28  7 65 47 39  0
 46 59 44 41 37 36 14 34 18 43 42 17 21 45 57 30 69 56 33 50 64 63]
Cost of the best solution: 31948.952732592617
Average time per generation: 0.38399060567220056


In [ ]:
# Run 2: population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=100, max_gen=6, elite_count=3, mutation_prob=0.02)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3275.2319476534162
Generation 2: Best Cost = 3275.2319476534162
Generation 3: Best Cost = 3275.2319476534162
Generation 4: Best Cost = 3275.2319476534162
Generation 5: Best Cost = 3275.2319476534162
Generation 6: Best Cost = 3275.2319476534162
Best solution found: [16 67 23 68 22 69 30  0 11 31 53 38 55 28 19 42 51  7 26 60 32 24  8 56
 46 20 15  4  9 35 27 25 13 39 49 54 29 34 64 37 66 59 44  2 21 41 47  5
 61 63 52  3 50 10 58 45 17 48 12 43  1 65 57  6 62 14 18 40 33 36]
Cost of the best solution: 32752.31947653416
Average time per generation: 0.10055979092915854


In [ ]:
# Run 3: population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=450, max_gen=6, elite_count=22, mutation_prob=0.045)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3025.2972756940417
Generation 2: Best Cost = 3025.2972756940417
Generation 3: Best Cost = 3025.2972756940417
Generation 4: Best Cost = 3025.2972756940417
Generation 5: Best Cost = 3025.2972756940417
Generation 6: Best Cost = 3025.2972756940417
Best solution found: [21 54 51 55 69 56 17 12 35 58  5 41 10 36  4 68 34  9 66 49 67 50 29 47
 44 31 20 19 14  3 18 15 33 60 62 45  7 11 39  2 48  1 52 40 25 65 32 59
 53 61 64 63 42 57 46 23 13  8 24 38 26 30 43 16 27 28 22  0  6 37]
Cost of the best solution: 30252.972756940417
Average time per generation: 0.48059701919555664


In [ ]:
# Run 4: population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=350, max_gen=6, elite_count=17, mutation_prob=0.035)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3144.092634306701
Generation 2: Best Cost = 3144.092634306701
Generation 3: Best Cost = 3144.092634306701
Generation 4: Best Cost = 3144.092634306701
Generation 5: Best Cost = 3144.092634306701
Generation 6: Best Cost = 3144.092634306701
Best solution found: [64 28 57 46 22 16 38 52 32 29 13  5 25 42 67  0 12 59  7  6 17 11 34 48
 35 23 27 45 10 26 24  1 58 56 62  4 31  8 44 20 60 36 51 54  2 19 50 63
 18 43  3 66 68 53 55 61 47 69 65 49  9 15 41 30 14 39 40 33 37 21]
Cost of the best solution: 31440.926343067007
Average time per generation: 0.30052383740743


In [ ]:
# Run 5: population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=250, max_gen=6, elite_count=12, mutation_prob=0.025)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3274.617565365627
Generation 2: Best Cost = 3274.617565365627
Generation 3: Best Cost = 3274.617565365627
Generation 4: Best Cost = 3274.617565365627
Generation 5: Best Cost = 3274.617565365627
Generation 6: Best Cost = 3274.617565365627
Best solution found: [24 42  9 64 68 16  5  6 54 25 27 40 53 39 61 34 13 36 15 48  7  0 37 51
 41 22 47 44 69 26 38  3 29 12 67 11 19 56 18 10  4 33 66 60 31 49 57 14
 65 46 55 43 20 45 32 23 35 30 28 17 21 63 59 52 62  1  2 58 50  8]
Cost of the best solution: 32746.175653656268
Average time per generation: 0.3529783089955648


In [ ]:
# Run 6: population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=150, max_gen=6, elite_count=30, mutation_prob=0.06)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3071.6612294238857
Generation 2: Best Cost = 3071.6612294238857
Generation 3: Best Cost = 3071.6612294238857
Generation 4: Best Cost = 3071.6612294238857
Generation 5: Best Cost = 3071.6612294238857
Generation 6: Best Cost = 3071.6612294238857
Best solution found: [25 13 29 33 10 40 32 43 39 52 60 44 18 31  5 65 67 38 45 16 42 46 36 23
  3 63 49 53 56  0 51  6 15 58 68 37 21  4 24 62 34 22 20  8 14 11 59  7
 50 17 30 61 55  9 35 12  1 64 28 69 41 19 54 48 66  2 47 57 26 27]
Cost of the best solution: 30716.612294238857
Average time per generation: 0.12979586919148764


In [ ]:
# Run 7: population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=500, max_gen=6, elite_count=15, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3155.4285992346186
Generation 2: Best Cost = 3155.4285992346186
Generation 3: Best Cost = 3155.4285992346186
Generation 4: Best Cost = 3155.4285992346186
Generation 5: Best Cost = 3155.4285992346186
Generation 6: Best Cost = 3155.4285992346186
Best solution found: [59 64 32 17 60 16 67 20 40 52 66 53 55 29 51 49 61 58 57 47  6 56 45  8
 26  7 48 27  3 54 19 30 23 21 11  4 28 68 63 15  9 31 33 38 14 18  1 35
 13 34 12 10 43 62 65 69 37 41 50 24 36  2  5 22 39 42  0 46 44 25]
Cost of the best solution: 31554.285992346187
Average time per generation: 0.4603521029154459


In [ ]:
# Run 8: population_size=300, max_gen=6, elite_count=20, mutation_prob=1
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=300, max_gen=6, elite_count=20, mutation_prob=1)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3057.154440415776
Generation 2: Best Cost = 3057.154440415776
Generation 3: Best Cost = 3057.154440415776
Generation 4: Best Cost = 3057.154440415776
Generation 5: Best Cost = 3057.154440415776
Generation 6: Best Cost = 3057.154440415776
Best solution found: [26 48 33 36 53 42  3  5 65 56 62 46 27 13 63 47 10 16 24 67 12 11 34 22
 35  4  9 38  6 51 37 23 41 55 57  8 25 17  1 19 68 52 20 32 39 43 21 54
 44 66 18 45 40 59 14 30 69  0 28 50 49 60 61  7  2 31 64 15 58 29]
Cost of the best solution: 30571.54440415776
Average time per generation: 1.1059296131134033


In [ ]:
# Run 9: population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=600, max_gen=6, elite_count=20, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3116.1413866654602
Generation 2: Best Cost = 3116.1413866654602
Generation 3: Best Cost = 3116.1413866654602
Generation 4: Best Cost = 3116.1413866654602
Generation 5: Best Cost = 3116.1413866654602
Generation 6: Best Cost = 3116.1413866654602
Best solution found: [60 17 30 49 36 61 47  3 45 51 68  0 52 18 48 53 59 39 43  7  5  1 21 58
 31 16 41 44 19  8 24 67 46 69 15 23 28 65 34 26 50 54  6 11  4 37 62  2
 40 22 12 63 27 25 10 33 35 14 42 56 13 32 66 29 38 57 55  9 64 20]
Cost of the best solution: 31161.413866654602
Average time per generation: 0.48948208491007483


In [ ]:
# Run 10: population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01
import numpy as np
import tsplib95
import random
import math
import time
from collections import namedtuple

# Load TSPLIB95 dataset
def load_tsplib95_instance(filename):
    problem = tsplib95.load(filename)
    if problem.is_depictable():
        nodes = np.array([problem.get_display(node) for node in problem.get_nodes()])
    else:
        print("This problem instance does not have display data.")
        nodes = None
    return nodes

# Calculate the distance matrix
def calculate_distance_matrix(nodes):
    n = len(nodes)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist_matrix[i, j] = np.linalg.norm(nodes[i] - nodes[j])
    return dist_matrix

# Define the BBO algorithm
class BBO:
    def __init__(self, dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01):
        self.dist_matrix = dist_matrix
        self.population_size = population_size
        self.max_gen = max_gen
        self.elite_count = elite_count
        self.mutation_prob = mutation_prob
        self.population = []
        self.elite = []
        self.best_solution = None
        self.best_cost = float('inf')

    def initialize_population(self):
        for _ in range(self.population_size):
            individual = np.random.permutation(len(self.dist_matrix))
            cost = self.calculate_cost(individual)
            self.population.append((individual, cost))

    def calculate_cost(self, individual):
        cost = 0
        for i in range(1, len(individual)):
           cost += self.dist_matrix[individual[i - 1], individual[i]]
        # Adding the cost to return to the starting point to complete the cycle
        cost += self.dist_matrix[individual[-1], individual[0]]
        return cost

    def calculate_latency(self, individual):
        latency = 0
        total_latency = 0
        for i in range(1, len(individual)):
            latency += self.dist_matrix[individual[i - 1], individual[i]]
            total_latency += latency
        return total_latency

    def migrate(self, emigrate, immigrate):
        for i in range(len(immigrate)):
            if random.random() < self.mutation_prob:
                swap_idx = np.random.randint(0, len(emigrate))
                immigrate[i], immigrate[swap_idx] = immigrate[swap_idx], immigrate[i]
        return immigrate

    def optimize(self):
        start_time = time.time()
        self.initialize_population()
        self.population.sort(key=lambda x: x[1])

        for gen in range(self.max_gen):
            new_population = self.population[:self.elite_count]
            for i in range(self.elite_count, self.population_size):
                emigrate_idx = random.randint(0, self.elite_count - 1)
                immigrate_idx = random.randint(self.elite_count, self.population_size - 1)
                emigrate = self.population[emigrate_idx][0].copy()
                immigrate = self.population[immigrate_idx][0].copy()
                new_individual = self.migrate(emigrate, immigrate)
                new_cost = self.calculate_latency(new_individual)
                new_population.append((new_individual, new_cost))

            self.population = new_population
            self.population.sort(key=lambda x: x[1])

            if self.population[0][1] < self.best_cost:
                self.best_cost = self.population[0][1]
                self.best_solution = self.population[0][0]
            print(f"Generation {gen+1}: Best Cost = {self.best_cost}")

        end_time = time.time()
        avg_time = (end_time - start_time) / self.max_gen

        return self.best_solution, self.best_cost * 10, avg_time * 10

# Load the dataset
filename = r'/content/drive/MyDrive/TSPLIB95 folder/st70.tsp'
nodes = load_tsplib95_instance(filename)
dist_matrix = calculate_distance_matrix(nodes)

# Run BBO algorithm
bbo = BBO(dist_matrix, population_size=700, max_gen=6, elite_count=500, mutation_prob=0.01)
best_solution, best_cost, avg_time = bbo.optimize()

print("Best solution found:", best_solution)
print("Cost of the best solution:", best_cost)
print("Average time per generation:", avg_time)

Generation 1: Best Cost = 3117.2003762081113
Generation 2: Best Cost = 3117.2003762081113
Generation 3: Best Cost = 3117.2003762081113
Generation 4: Best Cost = 3117.2003762081113
Generation 5: Best Cost = 3117.2003762081113
Generation 6: Best Cost = 3117.2003762081113
Best solution found: [10 20  1 26 19 51 39 24  5 33 59 44  9 16 41 63 38 13 25 31 35 60 37  3
 56 64 55 54 43 57 67 18 65 48 66  4  2 42 45 23 50 14 40 58 29  6 34 69
 22 21 15 28 52 62 49 53 61 12 30 68 17 27  7  0 36 46 47 32  8 11]
Cost of the best solution: 31172.003762081113
Average time per generation: 0.23583014806111652
